In [1]:
import numpy as np
from face_geometry import cpp_compare, project_xy,\
                          change_handedness, estimate_scale,\
                          move_and_rescale_z, unproject_xy,\
                          canonical_metric_landmarks, landmark_weights,\
                          get_metric_landmarks,\
                          DEBUG, PCF

In [2]:
# save a file with your face from your camera with as frame.jpeg, then run jpeg_to_landmark.py
# thereafter run this cell, then the cpp script (main.cpp), and then the next cell
landmarks = np.load('landmarks.npy')
np.save('canonical_metric_landmarks.npy', canonical_metric_landmarks)
np.save('landmark_weights.npy', np.stack((landmark_weights, landmark_weights)))

In [3]:
pcf = PCF()

# this line DEBUG.get_debug() is False
DEBUG.toggle()

cpp_compare("canonical_metric_landmarks", canonical_metric_landmarks)
cpp_compare("landmarks", landmarks)

DEBUG.toggle()

screen_landmarks = landmarks.copy()

screen_landmarks = project_xy(screen_landmarks, pcf)
depth_offset = np.mean(screen_landmarks[2, :])

intermediate_landmarks = screen_landmarks.copy()
intermediate_landmarks = change_handedness(intermediate_landmarks)
first_iteration_scale = estimate_scale(intermediate_landmarks)
print("first_iteration_scale:", first_iteration_scale)

DEBUG.toggle()

intermediate_landmarks = screen_landmarks.copy()
intermediate_landmarks = move_and_rescale_z(pcf, depth_offset, first_iteration_scale, intermediate_landmarks)
intermediate_landmarks = unproject_xy(pcf, intermediate_landmarks)
intermediate_landmarks = change_handedness(intermediate_landmarks)
second_iteration_scale = estimate_scale(intermediate_landmarks)
print("second_iteration_scale:", second_iteration_scale)

metric_landmarks = screen_landmarks.copy()
total_scale = first_iteration_scale * second_iteration_scale
metric_landmarks = move_and_rescale_z(pcf, depth_offset, total_scale, metric_landmarks)
metric_landmarks = unproject_xy(pcf, metric_landmarks)
metric_landmarks = change_handedness(metric_landmarks)

cpp_compare("metric_landmarks", metric_landmarks)

DEBUG.toggle()

metric_landmarks_2 = get_metric_landmarks(landmarks.copy(), pcf)

print("Check get_metric_landmarks function", np.sum(np.abs(metric_landmarks_2-metric_landmarks)**2))

canonical_metric_landmarks: 1.490073712786996e-11

landmarks: 0.0

first_iteration_scale: 0.017646663004658537
sources: 1.490073712786996e-11

targets: 6.770664287423341e-08

weighted_sources: 8.256299453950529e-14

weighted_targets: 1.5652631114705495e-10

total_weight logged: 1.0550000043585919

source_center_of_mass logged: [ 1.05234409e-16 -2.62520349e-01  3.74073921e+00]

centered_weighted_sources: 7.335573533966446e-13

design_matrix: 2.1706071575454937e-09

design_matrix_norm logged: 25.378533000998015

postrotation: 2.173713204800653e-12

prerotation: 1.7838146683446466e-13

rotation: 3.3478762473671e-12

scale logged: 1.0229649418240596

pointwise_diffs: 1.5086463967007842e-10

weighted_pointwise_diffs: 8.388228385885382e-12

translation logged: [ -1.60436787  -2.18943399 -53.17587781]

transform_mat: 8.952414440090347e-12

second_iteration_scale: 1.0229649418240592
metric_landmarks: 2.054501486150982e-06

Check get_metric_landmarks function 0.0
